In [ ]:
import pickle
import random

import glob
import math

# files = glob.glob('traces/proven*')
# files = glob.glob('traces/failed*')
# files = glob.glob('traces_updown/*')
files = glob.glob('traces_2023-10-23_11:33/failed_is_lub.exists_between_sub_self.pk')

traces = []
for file in files:
    with open(file, "rb") as f:
        trace = pickle.load(f)
        traces.append(trace)

len(files)

In [ ]:
# from goal_model_step.model import StepGoalModel
# import torch

# goal_path = 'goal_step.ckpt'
# goal_model = StepGoalModel.load(goal_path, device=torch.device("cuda"), freeze=True)


In [ ]:
# traces[11].status

In [ ]:
# for i in range(len(traces)):
#     print (traces[i].tree.distance_to_proof, goal_model.batch_generate(['<extra_id_0>' + traces[i].tree.goal]))

In [ ]:
def add_data(node, visits):
    data = []
    # proving_edges = [edge for edge in node.out_edges if edge.distance_to_proof() < math.inf]
    # for edge in proving_edges:
    if node.out_edges:
        for edge in node.out_edges:
            datum = {
                'goal': node.goal,
                'tactic': edge.tactic,
                'distance_to_proof': edge.distance_to_proof(),
                'visits': visits[node.goal],
            }
            # todo could record error message for e.g. self-correcting proof approach>
            if len(edge.dst) == 1 and isinstance(edge.dst[0], ErrorNode):
                datum['error'] = True
            else:
                datum['error'] = False
            data.append(datum)
    return data


full_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        full_data.extend(add_data(nodes[node], updated_visit_count))


In [ ]:
len(full_data)

In [ ]:
full_data[5]

In [ ]:
# add goal objective as in Polu et al
def add_goal_step_data(node, visits, max_proof_len=10, neg_threshold=2048):
    steps = node.distance_to_proof

    if steps == math.inf:
        if visits[node.goal] >= neg_threshold:
            target = 0
        else:
            return None
    elif steps >= max_proof_len:
        target = 1
    else:
        # linearly project steps so that shorter proofs are allocated to higher buckets
        # target = (max_bucket + 1 - steps) // (max_bucket // buckets)
        target = (max_proof_len + 1) - steps

    return {'goal': node.goal, 'target': target}




goal_step_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        step_datum = add_goal_step_data(nodes[node], updated_visit_count)
        if step_datum:
            goal_step_data.append(step_datum)

In [ ]:
def add_edge_data(node, visits, max_proof_len=9, neg_threshold=2048):
    data = []
    # proving_edges = [edge for edge in node.out_edges if edge.distance_to_proof() < math.inf]
    # for edge in proving_edges:
    if node.out_edges:
        for edge in node.out_edges:
            # if error, then give edge the worst score
            if len(edge.dst) == 1 and isinstance(edge.dst[0], ErrorNode):
                target = max_proof_len + 2
            elif edge.distance_to_proof() == math.inf:
                if sum([visits[d.goal] for d in edge.dst]) >= neg_threshold:
                    target = max_proof_len + 1
                else:
                    continue
            else:
                target = min(max_proof_len, (edge.distance_to_proof() - 1))

            datum = {
                'goal': node.goal,
                'tactic': edge.tactic,
                'target': target
            }

            data.append(datum)

    return data


edge_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        edge_data.extend(add_edge_data(nodes[node], updated_visit_count))


In [ ]:
len(edge_data)

In [ ]:
import numpy as np
np.unique([t['target'] for t in edge_data], return_counts=True )

In [ ]:
import matplotlib.pyplot as plt

plt.hist([t['target'] for t in edge_data])


In [ ]:
len(set([t['target'] for t in goal_step_data]))

In [ ]:
len(goal_step_data)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist([g['target'] for g in goal_step_data])

In [ ]:
import numpy as np
np.unique([t['target'] for t in goal_step_data], return_counts=True)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
np.arange(0, 11)

In [ ]:
weights = compute_class_weight(class_weight='balanced', classes=np.arange(0, 11), y=[t['target'] for t in goal_step_data] + [1,2,3,4]).tolist()

In [ ]:
weights

In [ ]:
import random
random.shuffle(goal_step_data)

with open("goal_step_data.pk", "wb") as f:
    pickle.dump(goal_step_data[:int(0.9 * len(goal_step_data))], f)

with open("goal_step_data_val.pk", "wb") as f:
    pickle.dump(goal_step_data[int(0.9*len(goal_step_data)):], f)
#
# with open("tac_data.pk", "wb") as f:
#     pickle.dump(full_data, f)



In [ ]:
from updown.search_tree_updown import ErrorNode, Status

goal_data = []

for trace in traces:
    nodes = trace.nodes

    updated_visit_count = {node: nodes[node].visit_count for node in nodes}

    for goal, node in nodes.items():
        for a in node.ancestors:
            updated_visit_count[a] += node.visit_count

    for node in nodes:
        node_ = nodes[node]
        goal_datum = {
            'goal': node,
            'raw_visit_count': node_.visit_count,
            'full_visit_count': updated_visit_count[node],
            'proved': int(node_.status==Status.PROVED),
            'distance_to_proof': node_.distance_to_proof
        }

        # goal_data.append((node, node_.visit_count, updated_visit_count[node], int(node_.status==Status.PROVED), node_.distance_to_proof))
        goal_data.append(goal_datum)



In [ ]:
goal_data[0]

In [ ]:
len(goal_data)

In [ ]:
len(goal_data)

In [ ]:
print (goal_data[2])

In [ ]:
print (goal_data[0])

In [ ]:
print (full_data[1])

In [ ]:
import random
random.shuffle(goal_data)

with open("goal_data.pk", "wb") as f:
    pickle.dump(goal_data[:int(0.9 * len(goal_data))], f)

with open("goal_data_val.pk", "wb") as f:
    pickle.dump(goal_data[int(0.9*len(goal_data)):], f)

with open("tac_data.pk", "wb") as f:
    pickle.dump(full_data, f)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
proven_visits = [g[2] for g in goal_data if g[3] == 1]

In [ ]:
len(proven_visits)

In [ ]:
len([a for a in proven_visits if a > 0]) / len(proven_visits)

In [ ]:
thresholds = [64, 128, 256, 512, 1024, 2056]

for threshold in thresholds:
    print (threshold)
    p = len([a for a in proven_visits if a > threshold]) / len(proven_visits)
    failed_visits = [g[2] for g in goal_data if g[3] != 1] #and g[2] < 1000]
    q = len([a for a in failed_visits if a > threshold]) / len([a for a in failed_visits if a > 0])
    # print (p/q)
    print (p, q)
    print ('\n')



In [ ]:
plt.hist(proven_visits, bins=100)
plt.show()

In [ ]:
failed_visits = [g[2] for g in goal_data if g[3] != 1] #and g[2] < 1000]

In [ ]:
# only ~7% of nodes are explored
len([a for a in failed_visits if a > 0]) / len(failed_visits)

In [ ]:
len([a for a in failed_visits if a > 512]) / len([a for a in failed_visits if a > 0])

In [ ]:

len([a for a in failed_visits if a > 512])


In [ ]:
plt.hist(failed_visits, bins=100)
plt.show()

In [ ]:
from prover.search_tree_context import TreeError

for trace in traces:
    nodes = trace.nodes

    for node in nodes:
        node_ = nodes[node]
        if node_.out_edges:
            if all([edge.dst[0].status == Status.FAILED and len(edge.dst) == 1 and not isinstance(edge.dst[0].inner, TreeError) for edge in node_.out_edges ]):
                print ('failed')
                
                    
        # goal_data.append((node, node_.visit_count, updated_visit_count[node], int(node_.status==Status.PROVED), node_.distance_to_proof))




In [ ]:
proven = [d for d in full_data if d[2] < math.inf]

In [ ]:
failed = [d for d in full_data if d[2] == math.inf]

In [ ]:
len(failed)

In [ ]:
# large percentage of failed tactics result in an error. Can safely take these as negative/low reward examples
len([f for f in failed if f[-1]]) / len(failed)

In [ ]:
len(proven) / len(full_data)

In [ ]:
prov_dict = {}

for d in proven:
    if d[0] not in prov_dict:
        prov_dict[d[0]] = [(d[1], d[2], d[3], d[4])]
    else:
        prov_dict[d[0]].append((d[1], d[2], d[3], d[4]))

In [ ]:
len(prov_dict)


In [ ]:
len(proven)

In [ ]:
# prov_dict.keys()

In [ ]:
# sort by largest visit_count (as proxy for difficulty) and then by distance_to_proof, then by time taken for tactic
sorted([sorted(l, key=lambda x: (x[1], x[3])) for l in list(prov_dict.values())], key= lambda l: l[0][2],reverse=True)

In [ ]:
# from render_proof import render_proof
# render_proof('traces/proven_set.nonempty.one_mem_div_2023-10-10_06:33.pk')
# from lean_dojo import *

# repo=LeanGitRepo(url='https://github.com/leanprover-community/mathlib', commit='32a7e535287f9c73f2e4d2aef306a39190f0b504')
# for trace in traces:
#     theorem = trace.theorem
#     proof = trace.proof
#     dojo, state_0 = Dojo(theorem).__enter__()
#     state_1 = dojo.run_tac(state_0, ', '.join(proof))
#     print (state_1)

